stm32f4 mikrokontrolcüsünde olan Basic Timer (TIM6) kullanılması ile ilgili örnek program.   
Bu programda Kart üzerindeki Mavi LED 0.2 saniye yanıp sönmektedir.

In [ ]:
#include "stm32f4xx.h"

int i=0;

void TIM6_DAC_IRQHandler(void){
    //Status Register
    TIM6->SR &= ~TIM_SR_UIF; //update interrupt flag
    if(i%2==0)
        GPIOD->ODR |= GPIO_ODR_OD15;
    else
        GPIOD->ODR &= ~GPIO_ODR_OD15;
    i++;
}


int main(){
    //TIMER 6 ve 7 APB1 busına bağlı olduğu için
    //RCC'nin APB1 busı ile ilgili yazmacının TIM6 
    //çevrim sinyalini aktif hale getiriyoruz.
    RCC->APB1ENR |= RCC_APB1ENR_TIM6EN;
    
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIODEN;
    GPIOD->MODER |= GPIO_MODER_MODE15_0;
    
    //TIMER 6'nın interruptını NVIC te aktifleştiriyoruz.
    NVIC_EnableIRQ(TIM6_DAC_IRQn);
    
    //DMA Interrupt Enable Reg
    TIM6->DIER |= TIM_DIER_UIE; //Update Interrupt Enable
    
    //Prescaler, önce yazmaç temizleniyor.   
    //Sonra yazmak istediğimiz değeri yazıyoruz.
    TIM6->PSC &= ~0xFFFFU;
    TIM6->PSC |= (0xFFFFU & 99U);
    
    //Autoreload register, önce yazmaç temizleniyor.   
    //Sonra yazmak istediğimiz değeri yazıyoruz.
    TIM6->ARR &= ~0xFFFFU;
    TIM6->ARR |= (0xFFFFU & 32000U);
    
    //Control Register 1
    TIM6->CR1 |= TIM_CR1_CEN; //Counter enable
    
    while(1);
    
    return 1;
}
